In [1]:
from sklearn import svm
import numpy as np, pandas as pd, os
from prettytable import PrettyTable
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
feature_names = list()
for index in range(1, 6):
    feature_names.extend(["Suit"+str(index), "Rank"+str(index)])

feature_names.append('class')


training_input_file = os.path.abspath('poker-hand-training-true.csv')
testing_input_file = os.path.abspath('poker-hand-testing.csv')

np.random.seed(350)     # seed for reproducible results

# To store configs
class myConfigs:
    features = 0
    classes = 0


config = myConfigs()

In [3]:
train_data = pd.read_csv(training_input_file, names=feature_names)
test_data = pd.read_csv(testing_input_file, names=feature_names)

# Get features of data
config.features = len(train_data.columns) - 1
config.classes = len(set(train_data['class']))

# Shuffle training data
train_data = train_data.sample(frac=1).reset_index(drop=True)

# Seperate data and classes
train_y = np.array(train_data['class'])
train_x = np.array(train_data.drop('class', 1))

test_y = np.array(test_data['class'])
test_x = np.array(test_data.drop('class', 1))

if __name__ == '__main__':
    tab = PrettyTable(['Config', 'Value'])
    configs = vars(config)

    for key in configs:
        tab.add_row([key, configs[key]])
    print(tab)

+----------+-------+
|  Config  | Value |
+----------+-------+
| features |   10  |
| classes  |   10  |
+----------+-------+


In [ ]:
model = svm.SVC()
print("Generated model! Going to train...")

# Training
model.fit(train_x, train_y)
print("Done training! Going to test...")

# Testing
testLen = len(test_x)
predictions = list()
for l in range(testLen):
    predictions.append(model.predict([test_x[l]])[0])
    if l == (testLen // 4) or l == (testLen // 2) or l == ((testLen*3) // 4):
        print("Quarter done")

print("Generated predictions!")

Generated model! Going to train...
Done training! Going to test...


In [ ]:
import math

avgError = sum([math.fabs(x-y) for x, y in zip(predictions, test_y)]) / len(predictions)
print("Average error :", avgError)

rmsError = sum([(x-y)**2 for x, y in zip(predictions, test_y)]) / len(predictions)
rmsError = math.sqrt(rmsError)
print("RMSE :", rmsError)

In [ ]:
plt.plot(test_y, 'd', label='actual')
plt.plot(predictions, 'yd', label='predicted')
plt.legend()
plt.show()

In [ ]:
preds = predictions
confMat = [[0] * config.classes for x in range(config.classes)]
acc = 0

for i in range(len(preds)):
    predLabel = preds[i]
    actLabel = test_y[i]
    confMat[actLabel][predLabel] += 1
    if predLabel == actLabel:
        acc += 1

header = [""]
for l in range(config.classes):
    header.append("Pred " + str(l))
    
tab = PrettyTable(header)
l = 0
for c in confMat:
    tab.add_row(["Real " + str(l)] + c)
    l += 1

print(tab)
print("Accuracy :", acc * 100 / len(preds))